## Lending Club - classification of loans

This project aims to analyze data for loans through 2007-2015 from Lending Club available on Kaggle. Dataset contains over 887 thousand observations and 74 variables among which one is describing the loan status. The goal is to create machine learning model to categorize the loans as good or bad. 

Project is split into two notebooks, this one is for data preparation and second for developing a model.

Contents:

    1. Preparing dataset for preprocessing
    2. Reviewing variables - drop and edit
    3. Missing values

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

import warnings
warnings.filterwarnings('ignore')

### 1. Preparing dataset for preprocessing

In this part I will load data, briefly review the variables and prepare the 'y' value that will describe each loan as good or bad.

In [2]:
data=pd.read_csv('loan.csv',parse_dates=True)
pd.set_option('display.max_columns', None)

In [3]:
data.shape

(887379, 74)

In [4]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pd.value_counts(data.loan_status).to_frame().reset_index()

,index,loan_status
0,Current,601779
1,Fully Paid,207723
2,Charged Off,45248
3,Late (31-120 days),11591
4,Issued,8460
5,In Grace Period,6253
6,Late (16-30 days),2357
7,Does not meet the credit policy. Status:Fully ...,1988
8,Default,1219
9,Does not meet the credit policy. Status:Charge...,761


There are 9 unique loan statuses. I will drop ones that are fully paid as these are historical entries. Next step will be to assign 0 (good) to Current loans and 1 (bad) to rest including: default and late loans, ones that were charged off or are in grace period.

First two are self-explanatory, charged off loan is a debt that is deemed unlikely to be collected by the creditor but the debt is not necessarily forgiven or written off entirely, a grace period is a provision in most loan contracts which allows payment to be received for a certain period of time after the actual due date.

In [6]:
data = data[data.loan_status != 'Fully Paid']
data = data[data.loan_status != 'Does not meet the credit policy. Status:Fully Paid']

In [7]:
data['rating'] = np.where((data.loan_status != 'Current'), 1, 0)

In [8]:
pd.value_counts(data.rating).to_frame()

,rating
0,601779
1,75889


In [9]:
print ('Bad Loan Ratio: %.2f%%'  % (data.rating.sum()/len(data)*100))

Bad Loan Ratio: 11.20%


The data is strongly imbalanced, however there are over 75 thousand bad loans that should suffice for a model to learn.

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677668 entries, 1 to 887378
Data columns (total 75 columns):
id                             677668 non-null int64
member_id                      677668 non-null int64
loan_amnt                      677668 non-null float64
funded_amnt                    677668 non-null float64
funded_amnt_inv                677668 non-null float64
term                           677668 non-null object
int_rate                       677668 non-null float64
installment                    677668 non-null float64
grade                          677668 non-null object
sub_grade                      677668 non-null object
emp_title                      636807 non-null object
emp_length                     677668 non-null object
home_ownership                 677668 non-null object
annual_inc                     677668 non-null float64
verification_status            677668 non-null object
issue_d                        677668 non-null object
loan_status          

### 2. Reviewing variables - drop and edit

In this part I will review each non-numerical variable to either edit or drop it.

There are two columns that describe a reason for the loan - title and purpose. As shown below title has many more categories which makes it less specific and helpful for the model, so it will be dropped.

In [11]:
pd.value_counts(data.title).to_frame()

,title
Debt consolidation,355393
Credit card refinancing,145252
Home improvement,34426
Other,27143
Major purchase,10240
Debt Consolidation,6484
Medical expenses,5597
Business,5503
Car financing,4720
Moving and relocation,3311


In [12]:
pd.value_counts(data.purpose).to_frame()

,purpose
debt_consolidation,402643
credit_card,163661
home_improvement,39026
other,31250
major_purchase,11786
small_business,6913
medical,6219
car,5614
moving,3780
vacation,3405


Application type variable shows whether the loan is individual or joint - number of joint loans will reflect huge number of NaN values in other variables dedicated for these loans.

Will change this variable to binary.

In [13]:
pd.value_counts(data.application_type).to_frame()

,application_type
INDIVIDUAL,677158
JOINT,510


In [14]:
app_type={'INDIVIDUAL':0,'JOINT':1}
data.application_type.replace(app_type,inplace=True)

In [15]:
pd.value_counts(data.term).to_frame()

,term
36 months,451761
60 months,225907


Term variable will be changed to numerical.

In [16]:
term={' 36 months':36,' 60 months':60}
data.term.replace(term,inplace=True)

Following two variables are dedicated to credit rating of each individual. Will change them to numerical while making sure that the hierarchy is taken into account. Lowest number will mean best grade/subgrade.

In [17]:
pd.value_counts(data.grade).to_frame()

,grade
C,192701
B,187720
D,109028
A,108433
E,57399
F,18166
G,4221


In [18]:
grade=data.grade.unique()
grade.sort()
grade

array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype=object)

In [19]:
for x,e in enumerate(grade):
    data.grade.replace(to_replace=e,value=x,inplace=True)

In [20]:
data.grade.unique()

array([2, 1, 5, 3, 0, 4, 6], dtype=int64)

In [21]:
pd.value_counts(data.sub_grade).to_frame()

,sub_grade
B4,40963
C1,40811
B3,40453
C2,40321
C3,39839
C4,39261
B5,36550
B2,35702
B1,34052
A5,33758


In [22]:
sub_grade=data.sub_grade.unique()
sub_grade.sort()
sub_grade

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1',
       'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2',
       'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [23]:
for x,e in enumerate(sub_grade):
    data.sub_grade.replace(to_replace=e,value=x,inplace=True)

data.sub_grade.unique()

array([13,  9, 14, 26, 10,  6,  8,  7, 11, 16,  4,  5, 23, 17, 28, 18, 19,
        1, 12,  2, 25, 15,  3, 24, 20, 21, 22, 32, 31, 30, 27, 29,  0, 33,
       34], dtype=int64)

Following two variables describe title and length of employment. Title has 212 thousand categories so it will be dropped. Lenghth of employment should be sufficient to show whether an individual has a stable job.

In [24]:
pd.value_counts(data.emp_title).to_frame()

,emp_title
Teacher,11875
Manager,9680
Owner,5028
Registered Nurse,4836
RN,4657
Supervisor,4297
Sales,3674
Project Manager,3406
Driver,3175
Office Manager,3022


In [25]:
pd.value_counts(data.emp_length).to_frame()

,emp_length
10+ years,227507
2 years,59076
< 1 year,53210
3 years,52986
1 year,42946
5 years,40726
4 years,38958
n/a,37434
8 years,34185
7 years,33043


In [26]:
emp_len={'n/a':0,'< 1 year':1,'1 year':2,'2 years':3,'3 years':4,'4 years':5,'5 years':6,'6 years':7,'7 years':8,'8 years':9,'9 years':10,'10+ years':11}
data.emp_length.replace(emp_len,inplace=True)
data.emp_length.unique()

array([ 1,  2,  9,  5,  4, 11, 10,  3,  8,  7,  6,  0], dtype=int64)

Home ownership variable should be informative for model as individuals who own their home should be much safer clients that ones that only rent it.

In [27]:
pd.value_counts(data.home_ownership).to_frame()

,home_ownership
MORTGAGE,337683
RENT,270560
OWN,69372
OTHER,41
NONE,10
ANY,2


Verification status variable indicated whether the source of income of a client was verified.

In [28]:
pd.value_counts(data.verification_status).to_frame()

,verification_status
Source Verified,269079
Verified,217016
Not Verified,191573


Payment plan variable will be dropped as it has only 3 'y' values.

In [29]:
pd.value_counts(data.pymnt_plan).to_frame()

,pymnt_plan
n,677660
y,8


Zip code information is to specific, there are 930 individual values, and there is no sense to make it more general as cutting it to two digits as this will only describe state, which does next veriable. Zip code will be dropped.

In [30]:
pd.value_counts(data.zip_code).to_frame()

,zip_code
750xx,7065
945xx,6976
112xx,6974
606xx,6505
300xx,6181
070xx,5618
331xx,5605
100xx,5403
770xx,5250
900xx,5213


In [31]:
pd.value_counts(data.addr_state).to_frame()

,addr_state
CA,93516
NY,56681
TX,54704
FL,46754
IL,27654
NJ,25389
PA,24462
OH,23280
GA,22362
VA,19688


Next variable is initial listing status of the loan. Possible values are – W, F and will be changed to binary.

In [32]:
pd.value_counts(data.initial_list_status).to_frame()

,initial_list_status
w,371954
f,305714


In [33]:
int_status={'w':0,'f':1}
data.initial_list_status.replace(int_status,inplace=True)

Policy code has only 1 value so will be dropped.

In [34]:
pd.value_counts(data.policy_code).to_frame()

,policy_code
1.0,677668


Recoveries variable informs about post charge off gross recovery. Will transform this to binary that will show whether this loan was recoveried. Will drop recovery fee as it is doubling similar information.

In [35]:
pd.value_counts(data.recoveries).to_frame()

,recoveries
0.00,652991
200.00,10
11.70,9
14.28,9
10.00,9
16.50,8
10.40,8
800.00,8
16.80,8
100.00,8


In [36]:
data['recovery'] = np.where((data.recoveries != 0.00), 1, 0)

In [37]:
pd.value_counts(data.collection_recovery_fee).to_frame()

,collection_recovery_fee
0.0000,654161
2.0000,18
1.8000,13
1.5500,12
1.2000,12
1.8800,12
4.5200,11
1.6000,11
1.0700,11
1.9000,11


There are couple variables that can be transformed to date time.

In [38]:
data.issue_d=pd.to_datetime(data.issue_d)

In [39]:
earliest_cr_line=pd.to_datetime(data.earliest_cr_line)
data.earliest_cr_line=earliest_cr_line.dt.year

In [40]:
data.last_pymnt_d=pd.to_datetime(data.last_pymnt_d)
data.next_pymnt_d=pd.to_datetime(data.next_pymnt_d)
data.last_credit_pull_d=pd.to_datetime(data.last_credit_pull_d)

Dropping all variables mentioned above.

In [41]:
data.drop(['id','member_id','desc','loan_status','url', 'title','collection_recovery_fee','recoveries','policy_code','zip_code','emp_title','pymnt_plan'],axis=1,inplace=True)

In [42]:
data.head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,rating,recovery
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,1,RENT,30000.0,Source Verified,2011-12-01,car,GA,1.00,0.0,1999.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,1,0.00,0.00,1008.71,1008.71,456.46,435.17,0.0,2013-04-01,119.66,NaT,2013-09-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,2,RENT,80000.0,Source Verified,2011-12-01,other,OR,17.94,0.0,1996.0,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,1,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.0,2016-01-01,67.79,2016-02-01,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
6,7000.0,7000.0,7000.0,60,15.96,170.08,2,14,9,RENT,47004.0,Not Verified,2011-12-01,debt_consolidation,NC,23.51,0.0,2005.0,1.0,NaN,NaN,7.0,0.0,17726.0,85.6,11.0,1,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0.0,2016-01-01,170.08,2016-02-01,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
8,5600.0,5600.0,5600.0,60,21.28,152.39,5,26,5,OWN,40000.0,Source Verified,2011-12-01,small_business,CA,5.55,0.0,2004.0,2.0,NaN,NaN,11.0,0.0,5210.0,32.6,13.0,1,0.00,0.00,646.02,646.02,162.02,294.94,0.0,2012-04-01,152.39,NaT,2012-08-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9,5375.0,5375.0,5350.0,60,12.69,121.45,1,9,1,RENT,15000.0,Verified,2011-12-01,other,TX,18.08,0.0,2004.0,0.0,NaN,NaN,2.0,0.0,9279.0,36.5,3.0,1,0.00,0.00,1476.19,1469.34,673.48,533.42,0.0,2012-11-01,121.45,NaT,2013-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
12,9000.0,9000.0,9000.0,36,13.49,305.38,2,10,1,RENT,30000.0,Source Verified,2011-12-01,debt_consolidation,VA,10.08,0.0,2004.0,1.0,NaN,NaN,4.0,0.0,10452.0,91.7,9.0,1,0.00,0.00,2270.70,2270.70,1256.14,570.26,0.0,2012-07-01,305.38,NaT,2012-11-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
14,10000.0,10000.0,10000.0,36,10.65,325.74,1,6,4,RENT,100000.0,Source Verified,2011-12-01,other,CA,7.06,0.0,1991.0,2.0,NaN,NaN,14.0,0.0,11997.0,55.5,29.0,1,0.00,0.00,7471.99,7471.99,5433.47,1393.42,0.0,2013-10-01,325.74,NaT,2014-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
21,21000.0,21000.0,21000.0,36,12.42,701.73,1,8,11,RENT,105000.0,Verified,2011-12-01,debt_consolidation,FL,13.22,0.0,1983.0,0.0,NaN,NaN,7.0,0.0,32135.0,90.3,38.0,1,0.00,0.00,14025.40,14025.40,10694.96,3330.44,0.0,2013-09-01,701.73,NaT,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
24,6000.0,6000.0,6000.0,36,11.71,198.46,1,7,2,RENT,76000.0,Not Verified,2011-12-01,major_purchase,CA,2.40,0.0,2001.0,1.0,NaN,NaN,7.0,0.0,5963.0,29.7,7.0,1,0.00,0.00,2050.14,2050.14,1305.58,475.25,0.0,2012-10-01,198.46,NaT,2013-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
26,15000.0,15000.0,8725.0,36,14.27,514.64,2,11,10,RENT,60000.0,Not Verified,2011-12-01,debt_consolidation,NY,15.22,0.0,2003.0,1.0,NaN,NaN,7.0,

### 3. Missing values

There are observations that contain missing values, I will review and transform them variable by variable.

Starting with defining a function to create a data frame of metadata containing count of null values and type.

In [43]:
def meta (dataframe):
    metadata = []
    for f in data.columns:
    
        # Counting null values
        null = data[f].isnull().sum()
    
        # Defining the data type 
        dtype = data[f].dtype
    
        # Creating a Dict that contains all the metadata for the variable
        f_dict = {
            'varname': f,
            'nulls':null,
            'dtype': dtype
        }
        metadata.append(f_dict)

    meta = pd.DataFrame(metadata, columns=['varname','nulls', 'dtype'])
    meta.set_index('varname', inplace=True)
    meta=meta.sort_values(by=['nulls'],ascending=False)
    return meta

In [44]:
meta(data)

,nulls,dtype
varname,,
dti_joint,677160,float64
annual_inc_joint,677158,float64
verification_status_joint,677158,object
il_util,659177,float64
mths_since_rcnt_il,656998,float64
inq_fi,656440,float64
total_cu_tl,656440,float64
all_util,656440,float64
max_bal_bc,656440,float64


Variables: dti_joint, annual_inc_joint and verification_status_joint have so many null values as there are only 510 joint loans. Will replace NaN with 0 and 'None' for status.

In [45]:
data.dti_joint=data.dti_joint.replace(np.nan,0)
data.annual_inc_joint=data.annual_inc_joint.replace(np.nan,0)
data.verification_status_joint=data.verification_status_joint.replace(np.nan,'None')

Investigating variables connected to open_acc_6m which shows number of open trades in last 6 months. Variables open_il_6m, open_il_12m, open_il_24m, mths_since_rcnt_il, total_bal_il, il_util, open_rv_12m, open_rv_24m, max_bal_bc, all_util, inq_fi, total_cu_tl, inq_last_12m, collections_12_mths_ex_med have null values for the same rows - I will change them all to 0 as missing vaules show lack of open trades. 

In [46]:
data.loc[(data.open_acc_6m.isnull())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656440 entries, 1 to 887378
Data columns (total 64 columns):
loan_amnt                      656440 non-null float64
funded_amnt                    656440 non-null float64
funded_amnt_inv                656440 non-null float64
term                           656440 non-null int64
int_rate                       656440 non-null float64
installment                    656440 non-null float64
grade                          656440 non-null int64
sub_grade                      656440 non-null int64
emp_length                     656440 non-null int64
home_ownership                 656440 non-null object
annual_inc                     656440 non-null float64
verification_status            656440 non-null object
issue_d                        656440 non-null datetime64[ns]
purpose                        656440 non-null object
addr_state                     656440 non-null object
dti                            656440 non-null float64
delinq_2yrs   

In [47]:
variables1=['open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m','collections_12_mths_ex_med']

for e in variables1:
    data[e]=data[e].replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
mths_since_last_record,567148,float64
mths_since_last_major_derog,494756,float64
mths_since_last_delinq,337650,float64
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
tot_coll_amt,14599,float64
total_rev_hi_lim,14599,float64
tot_cur_bal,14599,float64
revol_util,316,float64


Variables containing month since last occurence of specific action have plenty null values that I understand as lack of the occurence.

In [48]:
pd.value_counts(data.mths_since_last_record).unique()

array([1627, 1625, 1624, 1623, 1619, 1615, 1610, 1609, 1607, 1601, 1593,
       1586, 1553, 1530, 1517, 1505, 1496, 1492, 1483, 1473, 1468, 1460,
       1451, 1447, 1442, 1437, 1388, 1367, 1359, 1263, 1253, 1188, 1146,
       1129, 1127, 1121, 1119, 1093, 1090, 1089, 1087, 1077, 1059, 1043,
       1017,  998,  982,  972,  958,  936,  935,  929,  928,  921,  913,
        906,  905,  896,  884,  870,  869,  864,  862,  850,  849,  844,
        841,  839,  834,  833,  826,  820,  817,  815,  812,  809,  803,
        771,  738,  712,  695,  692,  684,  624,  600,  593,  541,  533,
        511,  471,  469,  412,  400,  395,  375,  370,  355,  340,  318,
        303,  286,  283,  281,  279,  234,  227,  210,  189,  156,  151,
        131,  107,   64,   53,    7,    1], dtype=int64)

In [49]:
pd.value_counts(data.mths_since_last_major_derog).unique()

array([2835, 2794, 2788, 2781, 2768, 2762, 2756, 2730, 2674, 2668, 2665,
       2662, 2644, 2625, 2624, 2621, 2616, 2602, 2588, 2585, 2573, 2572,
       2569, 2555, 2553, 2550, 2549, 2543, 2540, 2532, 2530, 2523, 2517,
       2511, 2502, 2496, 2479, 2458, 2454, 2425, 2416, 2389, 2385, 2310,
       2262, 2261, 2259, 2190, 2162, 2145, 2103, 2093, 2088, 2060, 2051,
       1998, 1989, 1968, 1913, 1840, 1683, 1598, 1570, 1545, 1495, 1491,
       1412, 1361, 1340, 1252, 1153, 1080,  839,  613,  599,  504,  497,
        216,  156,  135,  128,  117,  110,   96,   94,   89,   80,   78,
         76,   73,   71,   70,   69,   67,   66,   65,   63,   59,   57,
         56,   54,   52,   48,   46,   43,   42,   41,   39,   38,   37,
         35,   33,   32,   30,   27,   26,   25,   24,   22,   21,   20,
         19,   17,   16,   15,   14,   11,   10,    8,    7,    6,    5,
          4,    3,    2,    1], dtype=int64)

In [50]:
pd.value_counts(data.mths_since_last_delinq).unique()

array([6941, 6940, 6707, 6698, 6695, 6584, 6405, 6401, 6284, 5977, 5961,
       5920, 5796, 5747, 5691, 5622, 5612, 5509, 5441, 5359, 5325, 5310,
       5255, 5248, 4997, 4992, 4961, 4916, 4910, 4821, 4795, 4660, 4637,
       4541, 4459, 4435, 4406, 4274, 4260, 4196, 4168, 4147, 4107, 3975,
       3961, 3455, 3220, 2974, 2964, 2935, 2919, 2910, 2905, 2900, 2895,
       2889, 2844, 2830, 2828, 2827, 2820, 2810, 2803, 2778, 2775, 2768,
       2693, 2662, 2599, 2528, 2518, 2438, 2350, 2318, 2160, 2036, 2011,
       1931, 1841, 1711,  991,  808,  116,   47,   46,   33,   28,   27,
         23,   20,   19,   17,   16,   14,   13,   10,    9,    8,    7,
          6,    5,    4,    3,    2,    1], dtype=int64)

Null values in these columns can't be replaced with 0 as it would mean that the last occurence was very recent. My understanding of these variables is that the key information is whether the specific action took place (delinquency, public record, worse rating), so I will turn these into binary categories of Yes (1), No (0).

In [51]:
data.loc[(data.mths_since_last_delinq.notnull()),'delinq']=1
data.loc[(data.mths_since_last_delinq.isnull()),'delinq']=0

data.loc[(data.mths_since_last_major_derog.notnull()),'derog']=1
data.loc[(data.mths_since_last_major_derog.isnull()),'derog']=0

data.loc[(data.mths_since_last_record.notnull()),'public_record']=1
data.loc[(data.mths_since_last_record.isnull()),'public_record']=0

data.drop(['mths_since_last_delinq','mths_since_last_major_derog','mths_since_last_record'],axis=1,inplace=True)

meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
total_rev_hi_lim,14599,float64
tot_coll_amt,14599,float64
tot_cur_bal,14599,float64
revol_util,316,float64
last_credit_pull_d,36,datetime64[ns]
inq_last_6mths,3,float64
acc_now_delinq,3,float64


Investigating tot_coll_amt, tot_cur_bal, total_rev_hi_lim - these are three totals that have missing values for the same observations. I will change them to 0 as they should mean that the total is 0.

In [52]:
data.loc[(data.tot_coll_amt.isnull())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14599 entries, 1 to 230656
Data columns (total 64 columns):
loan_amnt                     14599 non-null float64
funded_amnt                   14599 non-null float64
funded_amnt_inv               14599 non-null float64
term                          14599 non-null int64
int_rate                      14599 non-null float64
installment                   14599 non-null float64
grade                         14599 non-null int64
sub_grade                     14599 non-null int64
emp_length                    14599 non-null int64
home_ownership                14599 non-null object
annual_inc                    14599 non-null float64
verification_status           14599 non-null object
issue_d                       14599 non-null datetime64[ns]
purpose                       14599 non-null object
addr_state                    14599 non-null object
dti                           14599 non-null float64
delinq_2yrs                   14596 non-null fl

In [53]:
variables2=['tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']

for e in variables2:
    data[e]=data[e].replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
revol_util,316,float64
last_credit_pull_d,36,datetime64[ns]
pub_rec,3,float64
delinq_2yrs,3,float64
acc_now_delinq,3,float64
total_acc,3,float64
open_acc,3,float64


Variable revol_util is revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

In [54]:
data.loc[(data.revol_util.isnull())].head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,rating,recovery,delinq,derog,public_record
4730,6100.0,6100.0,6100.00000,36,14.27,209.29,2,11,0,MORTGAGE,23748.0,Verified,2011-11-01,home_improvement,MO,23.95,0.0,2002.0,0.0,2.0,0.0,0.0,NaN,4.0,1,0.0,0.0,1755.22,1755.22,989.91,471.90,0.000000,2012-06-01,209.29,NaT,2012-11-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
12076,3000.0,3000.0,3000.00000,60,22.11,83.05,6,30,7,OWN,17652.0,Source Verified,2011-07-01,other,MN,0.00,0.0,2008.0,0.0,3.0,0.0,0.0,NaN,5.0,1,0.0,0.0,498.49,498.49,113.96,217.52,0.000000,2011-11-01,83.05,NaT,2012-04-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
13930,2000.0,2000.0,1900.00000,36,18.39,72.70,4,21,2,RENT,24000.0,Not Verified,2011-06-01,other,AZ,22.60,0.0,2006.0,3.0,3.0,0.0,0.0,NaN,16.0,1,0.0,0.0,89.79,89.79,0.00,0.00,0.000000,NaT,0.00,NaT,2011-11-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
18045,20000.0,20000.0,4575.00344,60,20.48,535.24,6,31,11,RENT,42000.0,Verified,2011-03-01,debt_consolidation,OH,18.91,0.0,1996.0,2.0,3.0,0.0,0.0,NaN,18.0,1,0.0,0.0,6375.06,1452.03,2091.72,3252.38,0.000000,2012-02-01,535.24,NaT,2012-07-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
25329,8000.0,8000.0,8000.00000,36,15.58,279.61,3,17,2,OWN,38400.0,Verified,2010-10-01,major_purchase,PA,18.38,0.0,2005.0,0.0,8.0,0.0,0.0,NaN,11.0,1,0.0,0.0,6353.93,6353.93,4431.52,1705.38,0.000000,2012-08-01,279.61,NaT,2013-02-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
26046,3200.0,3200.0,3200.00000,36,17.93,115.58,4,24,6,RENT,15600.0,Verified,2010-09-01,debt_consolidation,MD,11.85,0.0,2006.0,0.0,2.0,0.0,0.0,NaN,5.0,1,0.0,0.0,937.45,937.45,569.79,352.70,14.959768,2011-05-01,115.58,NaT,2016-01-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0
26882,2100.0,2100.0,2100.00000,36,15.21,73.02,3,16,1,RENT,14400.0,Not Verified,2010-08-01,home_improvement,WI,20.33,0.0,2005.0,2.0,2.0,0.0,0.0,NaN,4.0,1,0.0,0.0,943.70,943.70,593.49,277.11,0.000000,2011-08-01,73.02,NaT,2012-01-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0
28754,7000.0,7000.0,6975.00000,60,18.67,180.32,5,26,5,RENT,27000.0,Source Verified,2010-06-01,other,DE,4.62,0.0,1997.0,1.0,5.0,0.0,0.0,NaN,12.0,1,0.0,0.0,1978.88,1971.82,0.00,0.00,0.000000,NaT,0.00,NaT,2016-01-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.0,0.0,0.0
29305,1500.0,1500.0,1500.00000,36,16.82,53.35,4,21,9,RENT,39996.0,Not Verified,2010-06-01,debt_consolidation,FL,13.05,0.0,2001.0,2.0,2.0,0.0,0.0,NaN,5.0,1,0.0,0.0,636.96,636.96,379.64,204.24,0.000000,2011-05-01,53.35,NaT,2011-10-01,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1.0,0.0,0.0
34337,1200.0,1200.0,1200.00000,36,14.26,41.17,2,14,1,RENT,4800.0,Not Verified,2009-11-01,other,TX,13.75,0.0,2002.0,0.0,7.0,0.0,0.0,NaN,9.0,1,0.0,0.0,1335.61,1335.61,165.55,80.83,14.978426,2010-05-01,41

In [55]:
pd.value_counts(data.revol_util).to_frame()

,revol_util
0.00,1946
59.00,1440
58.00,1419
61.00,1411
53.00,1408
52.00,1379
55.00,1370
57.00,1359
60.00,1357
62.00,1354


There is no clear answer to how to approach this variable, I will use 0 as this is the most common value and the amount of missing values is marginal.

In [56]:
data.revol_util=data.revol_util.replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
last_credit_pull_d,36,datetime64[ns]
open_acc,3,float64
delinq_2yrs,3,float64
acc_now_delinq,3,float64
total_acc,3,float64
pub_rec,3,float64
earliest_cr_line,3,float64


There are four datetime variables and three of them have missing values left. 

Variables last_credit_pull_d is the most recent month LC pulled credit for this loan, issue_d is the date loan was issued and next_payment_d is the date of next payment. There are not insightful variables so will be dropped.

I will check last_pymnt_d in more detail as this might have some predicitve value.

In [57]:
pd.value_counts(data.last_pymnt_d).to_frame()

,last_pymnt_d
2016-01-01,465495
2015-12-01,138809
2015-10-01,3805
2015-11-01,3407
2015-09-01,3230
2015-07-01,3137
2015-08-01,2970
2015-06-01,2623
2015-05-01,2366
2015-04-01,2195


In [58]:
late=data.loc[(data.last_pymnt_d=='2015-08-01')|(data.last_pymnt_d=='2015-09-01')|(data.last_pymnt_d=='2015-05-01')|(data.last_pymnt_d=='2015-06-01')]
pd.value_counts(late.rating).to_frame()

,rating
1,11189


This is clear information leak - model wouldn't have to learn, just check if last payment is late. I will transform this variable to binary category showing if any payment was received.

In [59]:
data.loc[(data.last_pymnt_d.notnull()),'pymnt_received']=1
data.loc[(data.last_pymnt_d.isnull()),'pymnt_received']=0

In [60]:
data.drop(['last_pymnt_d','issue_d','last_credit_pull_d','next_pymnt_d'],axis=1,inplace=True)

meta(data)

,nulls,dtype
varname,,
earliest_cr_line,3,float64
inq_last_6mths,3,float64
delinq_2yrs,3,float64
pub_rec,3,float64
total_acc,3,float64
open_acc,3,float64
acc_now_delinq,3,float64
open_acc_6m,0,float64
open_il_6m,0,float64


There are seven variables with 3 missing values, this is such a small number that I will just replace NaN with most common values.

In [61]:
variables3=['acc_now_delinq', 'open_acc', 'total_acc','pub_rec','delinq_2yrs','inq_last_6mths','earliest_cr_line']

for e in variables3:
    data[e]=data[e].replace(np.nan,data[e].mode()[0])
    
meta(data)

,nulls,dtype
varname,,
loan_amnt,0,float64
last_pymnt_amnt,0,float64
application_type,0,int64
annual_inc_joint,0,float64
dti_joint,0,float64
verification_status_joint,0,object
acc_now_delinq,0,float64
tot_coll_amt,0,float64
tot_cur_bal,0,float64


There are no more missing values, so I can proceed to setting up machine learning model.

In [62]:
data.to_csv('data.csv',index=False)